Elliot Ide Pozo

- <del>Crear script en python que permita extraer la data desde el server de base de datos (activismo.inf.uach.cl)</del>
- <del>Crear script en python que permita crear un json (dataset) de desafío, usuario, consulta, resultado (diccionario).</del>
- Crear un script para obtener la representación tf-idf (NLTK,,,).
- Crear un script que calcule al menos una métrica de las establecidas por cada conjunto.


In [1]:
import pandas as pd
import pymysql
from sshtunnel import SSHTunnelForwarder

In [2]:
# https://stackoverflow.com/questions/21903411/enable-python-to-connect-to-mysql-via-ssh-tunnelling
# https://stackoverflow.com/questions/42726681/conecting-to-mysql-in-a-remote-server-from-python

with SSHTunnelForwarder(
            'activismo.inf.uach.cl',
            ssh_username='vps238',
            ssh_password='vps238.2018',
            remote_bind_address=('127.0.0.1', 3306)) as tunnel:
        conn = pymysql.connect(host='127.0.0.1', user='learning',
                               passwd='analytic$',db='gonsa2',
                               port=tunnel.local_bind_port)
        query = "SELECT date_executed, FK_challenge_id_number, FK_student_nick,id_number,query,snippet \
        FROM Students_query\
        INNER JOIN Students_queries_results ON FK_student_query_id_number = id_number\
        WHERE FK_challenge_id_number = 1 AND FK_student_nick != 'test_student'\
        LIMIT 10";
        data = pd.read_sql(query, conn)
        display(data)
        
        conn.close()

,date_executed,FK_challenge_id_number,FK_student_nick,id_number,query,snippet
0,2019-04-29 14:39:53,1,eespinoza,11,vih en chile,<b>vih en chile</b>. Es urgente aumentar el di...
1,2019-04-29 14:39:53,1,eespinoza,11,vih en chile,"<b>En</b> <b>Chile</b>, la Ley 19.779 resguard..."
2,2019-04-29 14:39:53,1,eespinoza,11,vih en chile,¿Cuánto tarda una persona con <b>VIH</b> <b>en...
3,2019-04-29 14:39:53,1,eespinoza,11,vih en chile,<b>VIH</b> <b>Sida</b> <b>en</b> <b>Chile</b>:...
4,2019-04-29 14:39:53,1,eespinoza,11,vih en chile,<b>En</b> los últimos diez años los contagios ...
5,2019-04-29 14:39:53,1,eespinoza,11,vih en chile,Advierten que el <b>VIH</b> <b>sida</b> está d...
6,2019-04-29 14:39:53,1,eespinoza,11,vih en chile,<b>SIDA</b> <b>en</b> <b>Chile</b>: la epidemi...
7,2019-04-29 14:39:53,1,eespinoza,11,vih en chile,del tratamiento anti-<b>vih</b> para ayudar y ...
8,2019-04-29 14:39:53,1,eespinoza,11,vih en chile,Para el presidente de la Corporación <b>Sida</...
9,2019-04-29 14:39:53,1,eespinoza,11,vih en chile,Informe revela que <b>Chile</b> es el país don...


In [3]:
from bs4 import BeautifulSoup

In [4]:
#data['snippet'] = data['snippet'].apply(BeautifulSoup)

In [5]:
text = data['snippet'][0]
soup = BeautifulSoup(text)

print(soup.get_text())

vih en chile. Es urgente aumentar el diagnóstico de personas infectadas. La prevalencia de la infección en Latinoamérica no ha sido adecuadamente estudiada, pero, se calcula que en Chile la mitad de las personas que está infectada no ha sido diagnosticada y por lo tanto no está en control y tratamiento.
